In [1]:
!nvidia-smi


Tue May 27 21:20:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directo           c  tttttttt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "/kaggle/input/mimic-iv-merged/processed_dataset.csv"

# Load the latest version
df = pd.read_csv(file_path)
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas

In [3]:
df.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,note_id,input,target,input_tokens,target_tokens,hadm_id,note_type,note_seq,charttime,storetime
0,10000032,F,52,2180,2014 - 2016,2180-09-09,10000032-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1946,231,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00
1,10000032,F,52,2180,2014 - 2016,2180-09-09,10000032-DS-22,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Percoce...,"___ with HIV on HAART, HCV cirrhosis with asci...",2183,810,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00
2,10000117,F,48,2174,2008 - 2010,NaN,10000117-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> omepraz...,Ms. ___ is a ___ with history of GERD who pres...,1060,172,22927623,DS,21,2181-11-15 00:00:00,2181-11-15 15:04:00
3,10000117,F,48,2174,2008 - 2010,NaN,10000117-DS-22,<SEX> F <SERVICE> ORTHOPAEDICS <ALLERGIES> ome...,The patient presented to the emergency departm...,1195,330,27988844,DS,22,2183-09-21 00:00:00,2183-09-29 16:23:00
4,10000248,M,34,2192,2014 - 2016,NaN,10000248-DS-10,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,Mr. ___ is a ___ with history of mild FVIII de...,1961,230,20600184,DS,10,2192-11-30 00:00:00,2192-11-30 19:49:00


In [4]:
df['dod'] = df['dod'].apply(lambda x: 1 if pd.notnull(x) and str(x).strip() != '' else 0)

In [5]:
df = df.drop(columns=['subject_id','note_id','anchor_year','anchor_year_group','input_tokens','target_tokens','hadm_id','note_seq','charttime','storetime'])

In [6]:
# Add metadata to input text
def augment_input(row):
    mortality = "Deceased" if row['dod'] == 0 else "Alive"
    return (
        f"<Age> {row['anchor_age']} "
        f"<Mortality> {mortality} "
        f"<Note Type> {row['note_type']} "
        f"{row['input']}"
    )

df['augmented_input'] = df.apply(augment_input, axis=1)

In [7]:
df = df.drop(columns=[
'gender',	'anchor_age',	'dod',	'input',	'note_type'])

In [8]:
df.head()

,target,augmented_input
0,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
1,"___ with HIV on HAART, HCV cirrhosis with asci...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
2,Ms. ___ is a ___ with history of GERD who pres...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
3,The patient presented to the emergency departm...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
4,Mr. ___ is a ___ with history of mild FVIII de...,<Age> 34 <Mortality> Deceased <Note Type> DS <...


In [9]:
df

,target,augmented_input
0,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
1,"___ with HIV on HAART, HCV cirrhosis with asci...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
2,Ms. ___ is a ___ with history of GERD who pres...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
3,The patient presented to the emergency departm...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
4,Mr. ___ is a ___ with history of mild FVIII de...,<Age> 34 <Mortality> Deceased <Note Type> DS <...
...,...,...
270028,Mr. ___ is a ___ without significant medical h...,<Age> 57 <Mortality> Deceased <Note Type> DS <...
270029,Ms. ___ is a ___ year old female with a histor...,<Age> 46 <Mortality> Deceased <Note Type> DS <...
270030,___ year old man hx of epilepsy and left tempo...,<Age> 58 <Mortality> Alive <Note Type> DS <SEX...
270031,Mr. ___ was admitted with status epilepticus. ...,<Age> 58 <Mortality> Alive <Note Type> DS <SEX...


In [23]:
df['target'][0]

'___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week and confusion. # Ascites - p/w worsening abd distension and discomfort for last week. likely ___ portal HTN given underlying liver disease, though no ascitic fluid available on night of admission. No signs of heart failure noted on exam. This was ___ to med non-compliance and lack of diet restriction. SBP negative diuretics: > Furosemide 40 mg PO DAILY > Spironolactone 50 mg PO DAILY, chosen over the usual 100mg dose d/t K+ of 4.5. CXR was wnl, UA negative, Urine culture blood culture negative. Pt was losing excess fluid appropriately with stable lytes on the above regimen. Pt was scheduled with current PCP for ___ check upon discharge. Pt was scheduled for new PCP with Dr. ___ at ___ and follow up in Liver clinic to schedule outpatient screening EGD and ___. '

In [24]:
df['augmented_input'][0]

'<Age> 52 <Mortality> Alive <Note Type> DS <SEX> F <SERVICE> MEDICINE <ALLERGIES> No Known Allergies / Adverse Drug Reactions <ATTENDING> ___ <CHIEF COMPLAINT> Worsening ABD distension and pain <MAJOR SURGICAL OR INVASIVE PROCEDURE> Paracentesis <HISTORY OF PRESENT ILLNESS> ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone ___ weeks ago, because she feels like "they don\'t do anything" and that she "doesn\'t want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has

In [18]:
df.to_csv("final.csv", index=False)
